In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from dataclasses import asdict
from src import data_utils as d
from src import file_utils as f
from src.params import (
    CHARACTERISTICS_CSV_DIR,
    KS2_CSV_DIR
)
from src.constants import (
    CharacteristicsOriginalColumns, 
    CCIS_COLUMN_RENAME, 
    KS4_COLUMN_RENAME,
    KS2Columns,
    PupilDeprivationColumns
)

from glob import glob
import os
import uuid
from collections import defaultdict

In [21]:
df = d.load_csv("../data/raw/ks4_original_csv/ks4_original_sep20.csv", read_as_str=True)

Reading ../data/raw/ks4_original_csv/ks4_original_sep20.csv
Reading all data as str


In [22]:
list(df.columns)

['Candidate Number',
 'Forvus index number',
 'Surname',
 'Forename',
 'Gender',
 'Date of birth',
 'AGE',
 'Pupil Inclusion Status Flag',
 'Pupil included in progress 8 calculations',
 'DfE Establishment Number',
 'School URN',
 'UPN',
 'SEN',
 'Has the pupil been eligible for FSM in the last 6 years?',
 'Is the pupil looked after?',
 'Has the pupil been adopted from care?',
 'Is the pupil disadvantaged?',
 'EAL Group',
 'KS2 Prior Band',
 "Student's ethnicity",
 'Deprivation indicator - IDACI score',
 'Did the pupil join within the last 2 yrs?',
 'English Bacc?',
 'English & maths GCSEs A*-C?',
 'Entries',
 'GCSE Entries',
 'Entry English EBacc',
 'Entry Maths EBacc',
 'Entry Science EBacc',
 'Entry Humanities EBacc',
 'Entry Language EBacc',
 'Entry ALL EBacc',
 'Achieved English EBacc',
 'Achieved Maths EBacc',
 'Achieved Science EBacc',
 'Achieved Humanities EBacc',
 'Achieved Language EBacc',
 'Entry triple science',
 'Entry multiple languages',
 'KS2 English level (finely graded

In [23]:
ks2_columns = (set(asdict(KS2Columns).values()) | set(asdict(PupilDeprivationColumns).values()))
reverse_rename = {v: k for k, v in KS4_COLUMN_RENAME.items() if v in ks2_columns}
reverse_rename

{'deprivation_indicator_idaci_score': 'Deprivation indicator - IDACI score',
 'is_the_pupil_disadvantaged': 'Is the pupil disadvantaged?',
 'is_the_pupil_looked_after': 'Is the pupil looked after?',
 'ks2_english': 'KS2  ENGLISH (KS2)',
 'ks2_mathematics': 'KS2  MATHEMATICS (KS2)',
 'ks2_aps_va': 'KS2 APS (VA)',
 'ks2_aps_cubed': 'KS2 APS cubed',
 'ks2_aps_squared': 'KS2 APS squared',
 'ks2_english_ta': 'KS2 ENGLISH TA',
 'ks2_english_ps': 'KS2 English PS',
 'ks2_english_ps_deviation': 'KS2 English PS deviation',
 'ks2_english_ta_level': 'KS2 English TA level',
 'ks2_english_level_finely_graded': 'KS2 English level (finely graded)',
 'ks2_mathematics_ta': 'KS2 MATHEMATICS TA',
 'ks2_mathematics_ta_level': 'KS2 Mathematics TA level',
 'ks2_mathematics_level_finely_graded': 'KS2 Mathematics level (finely graded)',
 'ks2_maths_ps': 'KS2 Maths PS',
 'ks2_maths_ps_deviation': 'KS2 Maths PS deviation',
 'ks2_prior_band': 'KS2 Prior Band',
 'ks2_reading_ps': 'KS2 Reading PS',
 'ks2_reading_ta

In [24]:
cols_in_df = [col_name for col_name in reverse_rename.values() if col_name in df.columns]
cols_not_in_df = [col_name for col_name in reverse_rename.values() if col_name not in df.columns]
ks2_df = df[cols_in_df].head(5000)
for name in cols_not_in_df:
    ks2_df[name] = d.empty_series(len(ks2_df), index=ks2_df.index)
ks2_df

,Deprivation indicator - IDACI score,Is the pupil disadvantaged?,Is the pupil looked after?,KS2 English PS,KS2 English TA level,KS2 English level (finely graded),KS2 Mathematics TA level,KS2 Mathematics level (finely graded),KS2 Maths PS,KS2 Prior Band,...,KS2 APS cubed,KS2 APS squared,KS2 ENGLISH TA,KS2 English PS deviation,KS2 MATHEMATICS TA,KS2 Maths PS deviation,KS2 Reading PS,KS2 Reading TA level,KS2 Reading level (finely graded),KS2 mathematics (finely graded)
0,0,0,0,25,5,5.42,5,4.59,24,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,0,32,5,5,5,5.21,30,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,23,4,5.19,4,5.75,25,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,0,15,,5.35,4,5.79,14,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,0,23,4,4.83,2,4.38,21,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,1,0,19,3,5.45,,4.88,22,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4996,0,0,0,25,5,,5,,26,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,0,0,0,34,1,,5,5.29,33,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,0,1,0,31,5,2.5,3,5.33,30,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
ks2_df.to_csv(os.path.join(KS2_CSV_DIR, "ks2_original_sep20.csv"), index=False)